## Notebook calculating spectrum of $\Phi$ values for Haun and Tononi, 2019

#### First generate TPM

In [1]:
import numpy as np
import pyphi
import itertools
from pyphi import phi_spectrum


## First constuct the TPM
n_nodes = 8
n_states = 2**8


## Function to calculate probality that a given node is ON
def get_p_on(j,global_state,z=1/4.,n=5):
    if j==0:
        W_sum = global_state[j]+0.25*global_state[j+1]
    elif j == n_nodes-1:
        W_sum = global_state[j] + 0.25*global_state[j-1]
    else:
        W_sum = global_state[j] + 0.25*global_state[j-1] + 0.25*global_state[j+1]
    p_on = W_sum**n/(z+W_sum**n)
    return p_on    


## Create TPM
big_endian_states = itertools.product([0,1],repeat=n_nodes)

## Reverse the order so that the states are ordered in little-endian
all_states = []
for state in big_endian_states:
    all_states.append(state[::-1])

    
## Create a map between indices and states
state_index_map = {}
index = 0
for state in all_states:
    state_index_map[index]=state
    index = index+1
    
    
## For each state, get the probability of transitioning
TPM = np.zeros((n_states,n_states))
for i in range(n_states):
    current_state = state_index_map[i]
    print("Analyzing State = ",current_state)
        
    # Get probality of each node transitioning
    node_probs = []
    for j in range(n_nodes):
        p_on = get_p_on(j,current_state)
        p_off = 1 - p_on
        node_probs.append([p_off,p_on])
        
    # Get probability of global state transitions using node transitions
    for n in range(n_states):
        next_state = state_index_map[n]
        transition_prob = 1.0
        for m in range(n_nodes):
            ## get node_probs at the value of that state
            index = next_state[m]
            transition_prob = transition_prob*node_probs[m][index]
        TPM[i][n] = transition_prob
        
## Make sure transitions sum to unity
for k in range(n_states):
    if not np.isclose(np.sum(TPM[k][:]),1.0):
        print("ERROR TPM NOT SUMMING TO UNITY")
            
print("DONE")

Analyzing State =  (0, 0, 0, 0, 0, 0, 0, 0)
Analyzing State =  (1, 0, 0, 0, 0, 0, 0, 0)
Analyzing State =  (0, 1, 0, 0, 0, 0, 0, 0)
Analyzing State =  (1, 1, 0, 0, 0, 0, 0, 0)
Analyzing State =  (0, 0, 1, 0, 0, 0, 0, 0)
Analyzing State =  (1, 0, 1, 0, 0, 0, 0, 0)
Analyzing State =  (0, 1, 1, 0, 0, 0, 0, 0)
Analyzing State =  (1, 1, 1, 0, 0, 0, 0, 0)
Analyzing State =  (0, 0, 0, 1, 0, 0, 0, 0)
Analyzing State =  (1, 0, 0, 1, 0, 0, 0, 0)
Analyzing State =  (0, 1, 0, 1, 0, 0, 0, 0)
Analyzing State =  (1, 1, 0, 1, 0, 0, 0, 0)
Analyzing State =  (0, 0, 1, 1, 0, 0, 0, 0)
Analyzing State =  (1, 0, 1, 1, 0, 0, 0, 0)
Analyzing State =  (0, 1, 1, 1, 0, 0, 0, 0)
Analyzing State =  (1, 1, 1, 1, 0, 0, 0, 0)
Analyzing State =  (0, 0, 0, 0, 1, 0, 0, 0)
Analyzing State =  (1, 0, 0, 0, 1, 0, 0, 0)
Analyzing State =  (0, 1, 0, 0, 1, 0, 0, 0)
Analyzing State =  (1, 1, 0, 0, 1, 0, 0, 0)
Analyzing State =  (0, 0, 1, 0, 1, 0, 0, 0)
Analyzing State =  (1, 0, 1, 0, 1, 0, 0, 0)
Analyzing State =  (0, 1, 1, 0, 

#### Build network object and calculate $\Phi$

In [ ]:
# Set up network object
network = pyphi.Network(TPM, node_labels=['A','B','C','D','E','F','G','H'])
print("Network = ",network.node_labels)

# Put the system into an initial state
state = (0,0,0,0,0,0,0,0)
nodes = ['A','B','C','D','E','F','G','H']

## Get the requisite Subsystem
subsystem = pyphi.Subsystem(network, state, nodes)

## Calculate all Phi values
display_CES= False  # if True, output will display constellations
Phi_Spectrum = phi_spectrum.get_phi_spectrum(subsystem,display_CES)

print("\nCuts = ",Phi_Spectrum[0])
print("\nPhi Spectrum = ",Phi_Spectrum[1])

Network =  NodeLabels(('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'))


In [ ]:
Phi_MIP = phi_spectrum.get_Phi_MIP(Phi_Spectrum)
print("Phi MIP = ",Phi_MIP)